In [8]:
import serial
import serial.tools.list_ports
import pandas as pd
import time
import os

def select_serial_port():
    """Menampilkan daftar port serial yang tersedia dan meminta pengguna untuk memilih."""
    ports = serial.tools.list_ports.comports()
    print("Port serial yang tersedia:")
    
    if not ports:
        print("-> Tidak ada port serial yang ditemukan. Pastikan perangkat Anda terhubung.")
        return None

    for i, port in enumerate(ports):
        print(f"  {i + 1}: {port.device} - {port.description}")
    
    while True:
        try:
            choice = int(input("Pilih nomor port untuk ESP32 Anda: "))
            if 1 <= choice <= len(ports):
                return ports[choice - 1].device
            else:
                print("Pilihan tidak valid. Silakan pilih nomor dari daftar.")
        except (ValueError, IndexError):
            print("Input tidak valid. Harap masukkan nomor.")

def main():
    """
    Fungsi utama untuk menangkap data benchmark dari serial dan menyimpannya ke Excel.
    """
    # --- 1. Konfigurasi oleh Pengguna ---
    port = select_serial_port()
    if not port:
        return

    baud_rate = 115200  # Sesuai dengan Serial.begin(115200) di kode C++
    output_filename = input("Masukkan nama untuk file Excel keluaran (contoh: hasil_benchmark.xlsx): ")
    if not output_filename.lower().endswith('.xlsx'):
        output_filename += '.xlsx'
        
    print("\n--- Konfigurasi ---")
    print(f"  Port        : {port}")
    print(f"  Baud Rate   : {baud_rate}")
    print(f"  File Output : {output_filename}")
    
    # --- 2. Proses Penangkapan Data ---
    data_rows = []
    summary_data = {}
    column_headers = []
    is_reading_data = False

    try:
        # Menggunakan timeout untuk mencegah skrip macet jika tidak ada data masuk
        with serial.Serial(port, baud_rate, timeout=5) as ser:
            print("\nKoneksi berhasil. Menunggu data...")
            print(">>> Harap RESET papan ESP32 Anda untuk memulai proses benchmark. <<<")
            
            while True:
                line_bytes = ser.readline()
                if not line_bytes:
                    # Timeout terjadi, diasumsikan transmisi data selesai
                    print("\nTidak ada data diterima selama 5 detik. Proses dianggap selesai.")
                    break

                # Decode byte menjadi string dan bersihkan spasi
                line = line_bytes.decode('utf-8', errors='ignore').strip()
                if line:
                    print(f"Diterima: {line}")

                # Logika parsing untuk membedakan jenis baris
                if "Input\tExpected\tPredicted" in line:
                    column_headers = [h.strip() for h in line.split('\t')]
                    is_reading_data = True
                    continue
                
                if "====" in line:
                    is_reading_data = False
                    continue

                if is_reading_data:
                    # Ini adalah baris data tabel
                    # Memisahkan berdasarkan tab dan menghapus entri kosong dari tab ganda
                    values = [v.strip() for v in line.split('\t') if v.strip()]
                    if len(values) == len(column_headers):
                        # Konversi nilai ke tipe data numerik
                        typed_values = [float(v) for v in values]
                        data_rows.append(typed_values)
                    else:
                        print(f"Peringatan: Melewati baris data yang tidak sesuai format: {line}")

                # Parsing baris ringkasan
                if "Mean Squared Error (MSE):" in line:
                    mse_value = float(line.split(':')[1].strip())
                    summary_data["Metrik"] = ["Mean Squared Error (MSE)", "Rata-rata Waktu Inferensi (us)"]
                    summary_data["Nilai"] = [mse_value]

                if "Rata-rata waktu inferensi:" in line:
                    time_value_str = line.split(':')[1].strip().split(' ')[0]
                    time_value = float(time_value_str)
                    if "Nilai" in summary_data:
                        summary_data["Nilai"].append(time_value)
                    
                    # Benchmark selesai setelah baris ini, keluar dari loop
                    time.sleep(0.5) # Beri jeda singkat untuk memastikan semua data diterima
                    break

    except serial.SerialException as e:
        print(f"\nERROR: Tidak dapat membuka port {port}. {e}")
        return
    except Exception as e:
        print(f"\nTerjadi kesalahan tak terduga: {e}")
        return

    # --- 3. Proses dan Ekspor Data ke Excel ---
    if not data_rows or not column_headers:
        print("\nTidak ada data valid yang ditangkap. File Excel tidak akan dibuat.")
        return

    print("\nPenangkapan data selesai. Menyiapkan file Excel...")

    try:
        # Buat DataFrame dari data yang terkumpul
        df_data = pd.DataFrame(data_rows, columns=column_headers)
        df_summary = pd.DataFrame(summary_data)

        # Tulis DataFrame ke file Excel
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            df_data.to_excel(writer, sheet_name='Hasil Benchmark', index=False)
            
            # Tambahkan ringkasan di bawah tabel utama
            df_summary.to_excel(writer, sheet_name='Hasil Benchmark', index=False, startrow=len(df_data) + 2)
        
        print(f"\nBERHASIL! Data telah disimpan ke '{os.path.abspath(output_filename)}'")

    except Exception as e:
        print(f"\nERROR: Gagal menulis ke file Excel: {e}")

if __name__ == "__main__":
    main()

Port serial yang tersedia:
  1: COM3 - USB Serial Device (COM3)
  2: COM7 - Standard Serial over Bluetooth link (COM7)
  3: COM6 - Standard Serial over Bluetooth link (COM6)

--- Konfigurasi ---
  Port        : COM3
  Baud Rate   : 115200
  File Output : A_Int_4_RP2350.xlsx

Koneksi berhasil. Menunggu data...
>>> Harap RESET papan ESP32 Anda untuk memulai proses benchmark. <<<
Diterima: --- TFLite INT8 Model Benchmark (Case A, MSE) for RP2350/RP2040 ---
Diterima: Starting benchmark for INT8 model (Case A)...
Diterima: --------------------------------------------------
Diterima: Input	Expected	Predicted	Time (us)
Diterima: --------------------------------------------------
Peringatan: Melewati baris data yang tidak sesuai format: --------------------------------------------------
Diterima: 157.7500	316.5000		317.3287		2189
Diterima: 181.2500	363.5000		360.4228		1493
Diterima: -115.2500	-229.5000		-227.2230		1280
Diterima: -149.2500	-297.5000		-297.7405		1258
Diterima: -169.2500	-337.500